In [1]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 18.8 MB/s eta 0:00:00


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.7 MB/s eta 0:00:00


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 1. Loading the data

In [6]:
import torchvision
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
dataset = torchvision.datasets.ImageFolder('/content/gdrive/MyDrive/WikiArt')
dataset

Dataset ImageFolder
    Number of datapoints: 42500
    Root location: /content/gdrive/MyDrive/WikiArt

In [8]:
# Split the dataset into train set and validation + test set
# Stratified Sampling for train set and val_test set
train_index, val_test_index= train_test_split(
np.arange(len(dataset.targets)),
test_size=0.2, random_state=91,
shuffle=True,
stratify=dataset.targets)

In [9]:
# Split the val set and test set from val_test set:
test_index, val_index= train_test_split(
np.arange(len(val_test_index)),
test_size=0.5, random_state=91,
shuffle=True,
stratify=[dataset.targets[i] for i in val_test_index])

# 2. Preprocess the data

Huggingface models require 2 things: pixel_values and labels.

Preprocessing images typically comes down to (1) resizing them to a particular size (2) normalizing the color channels (R,G,B) using a mean and standard deviation. These are referred to as image transformations.

In addition, one typically performs what is called data augmentation during training (like random cropping and flipping) to make the model more robust and achieve higher accuracy. Data augmentation is also a great technique to increase the size of the training data. The following augmentation techniques are considered:

- RandomResizedCrop: This technique randomly crops and resizes the input image to a specified size. It helps in introducing variations in the scale and aspect ratio of the input images.
- RandomHorizontalFlip: This technique randomly flips the input image horizontally. The given image was randomly flipped horizontally at a preset probability of 50%
- RandomGrayscale: Convert image to grayscale. Because there is a considerable proportion of images are grayscale, our models need to be invariant to color variations, converting images to grayscale can help achieve this. By removing color information, the model focuses solely on the intensity values, making it less sensitive to changes in color distribution or lighting conditions.

In [10]:
from transformers import ViTImageProcessor
from torchvision.transforms.v2 import Compose, Normalize, Resize, ToTensor, RandomHorizontalFlip, RandomResizedCrop, RandomGrayscale, CenterCrop

In [11]:
# The model is pre-trained on ImageNet-21k, a dataset of 14 million labeled images
model_name = "google/vit-base-patch16-224-in21k"
image_processor = ViTImageProcessor.from_pretrained(model_name)
image_processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [12]:
image_mean, image_std = image_processor.image_mean, image_processor.image_std
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

In [13]:
normalize = Normalize(mean=image_mean, std=image_std)
train_transforms = Compose([
    RandomResizedCrop(crop_size),
    RandomHorizontalFlip(),
    RandomGrayscale(),
    ToTensor(),
    normalize])

val_test_transforms = Compose([
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [14]:
class TransformDataset(torch.utils.data.Dataset):
  def __init__(self, base_dataset, transformations):
    super(TransformDataset, self).__init__()
    self.base = base_dataset
    self.transformations = transformations

  def __len__(self):
    return len(self.base)

  def __getitem__(self, idx):
    x, y = self.base[idx]
    return self.transformations(x), y

In [15]:
# Subset dataset for train, test and val
train_subset = torch.utils.data.Subset(dataset, train_index)
test_subset = torch.utils.data.Subset(dataset, test_index)
val_subset = torch.utils.data.Subset(dataset, val_index)

In [16]:
# Apply data augmentation and transformation to the subsets:
train_dataset = TransformDataset(train_subset, train_transforms)
test_dataset = TransformDataset(test_subset, val_test_transforms)
val_dataset = TransformDataset(val_subset, val_test_transforms)

In [17]:
print('number of train images: {}'.format(len(train_dataset)))
print('number of test images: {}'.format(len(test_dataset)))
print('number of val images: {}'.format(len(val_dataset)))

number of train images: 34000
number of test images: 4250
number of val images: 4250


In [18]:
label2id = {}
id2label = {}

for i, class_name in enumerate(dataset.classes):
    label2id[class_name] = str(i)
    id2label[str(i)] = class_name

In [19]:
# Create a custom collator:
class ImageClassificationCollator:
    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor

    def __call__(self, batch):
        pixel_values =  torch.stack([x[0] for x in batch])
        labels = torch.tensor([x[1] for x in batch], dtype=torch.long)
        return {"pixel_values": pixel_values, "labels": labels}
collator = ImageClassificationCollator(image_processor)

In [20]:
# # create corresponding PyTorch DataLoaders for Pytorch Lightning:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, num_workers=8, shuffle=True, collate_fn=collator, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, num_workers=8, shuffle=False, collate_fn=collator, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, num_workers=8, shuffle=False, collate_fn=collator, pin_memory=True)

In [21]:
batch = next(iter(train_loader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k, v.shape)

pixel_values torch.Size([64, 3, 224, 224])
labels torch.Size([64])


In [22]:
next(iter(val_loader))['pixel_values'].shape

torch.Size([64, 3, 224, 224])

# 3. Define and train the model

In [24]:
import pytorch_lightning as pl
from transformers import ViTForImageClassification, AdamW
import torch.nn as nn

In [ ]:
# model = ViTForImageClassification.from_pretrained(
#     'google/vit-base-patch16-224-in21k',
#     num_labels=len(label2id),
#     label2id=label2id,
#     id2label=id2label
# )

In [28]:
class ViTLightningModule(pl.LightningModule):
    def __init__(self):
        super(ViTLightningModule, self).__init__()
        self.vit = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                              num_labels=len(id2label),
                                                              id2label=id2label,
                                                              label2id=label2id)

    def forward(self, pixel_values):
        outputs = self.vit(pixel_values=pixel_values)
        return outputs.logits

    def common_step(self, batch, batch_idx):
        pixel_values = batch['pixel_values']
        labels = batch['labels']
        logits = self(pixel_values)

        criterion = nn.CrossEntropyLoss()
        loss = criterion(logits, labels)
        predictions = logits.argmax(-1)
        correct = (predictions == labels).sum().item()
        accuracy = correct/pixel_values.shape[0]
        return loss, accuracy

    def training_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch, batch_idx)
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_acc", accuracy, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch, batch_idx)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", accuracy, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch, batch_idx)
        self.log("test_loss", loss, on_epoch=True, prog_bar=True)
        self.log("test_acc", accuracy, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        # We could make the optimizer more fancy by adding a scheduler and specifying which parameters do
        # not require weight_decay but just using AdamW out-of-the-box works fine
        return AdamW(self.parameters(), lr=2e-5)

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

    def test_dataloader(self):
        return test_loader

In [ ]:
# class Classifier(pl.LightningModule):

#     def __init__(self, model, lr: float = 2e-5, **kwargs):
#         super().__init__()
#         self.save_hyperparameters('lr', *list(kwargs))
#         self.model = model
#         self.forward = self.model.forward
#         self.val_acc = Accuracy(task='multiclass', num_classes=model.config.num_labels)
#         self.test_acc = Accuracy(task='multiclass', num_classes=model.config.num_labels)

#     def training_step(self, batch, batch_idx):
#         outputs = self(**batch)
#         self.log(f"train_loss", outputs.loss)
#         return outputs.loss

#     def validation_step(self, batch, batch_idx):
#         outputs = self(**batch)
#         self.log(f"val_loss", outputs.loss)
#         acc = self.val_acc(outputs.logits.argmax(1), batch['labels'])
#         self.log(f"val_acc", acc, prog_bar=True)
#         return outputs.loss

#     def validation_step(self, batch, batch_idx):
#         outputs = self(**batch)
#         self.log(f"test_loss", outputs.loss)
#         acc = self.test_acc(outputs.logits.argmax(1), batch['labels'])
#         self.log(f"val_acc", acc, prog_bar=True)
#         return outputs.loss

#     def configure_optimizers(self):
#         return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

In [26]:
early_stop_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    strict=True,
    verbose=True,
    mode='min')

In [29]:
pl.seed_everything(91)
model = ViTLightningModule()
trainer = pl.Trainer(accelerator='gpu', precision='16-mixed', callbacks=[early_stop_callback], max_epochs=15)
trainer.fit(model)

INFO:lightning_fabric.utilities.seed:Seed set to 91
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type                      | Params
---------------------------------------------------
0 |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.104 >= min_delta = 0.0. New best score: 0.291
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_loss did not improve in the last 2 records. Best score: 0.291. Signaling Trainer to stop.


In [30]:
trainer.test(ckpt_path='best')

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_1/checkpoints/epoch=11-step=6384.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_1/checkpoints/epoch=11-step=6384.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.86917644739151      │
│         test_loss         │    0.4053940176963806     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4053940176963806, 'test_acc': 0.86917644739151}]

# 4. Save checkpoints to Google Drive

In [31]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

In [32]:
folders_or_files_to_save= '/content/lightning_logs'
filename = 'ViT'

In [33]:
def save_file_to_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }
    media = MediaFileUpload(path,
                  mimetype='application/octet-stream',
                  resumable=True)
    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print('File ID: {}'.format(created.get('id')))
    return created

In [34]:
extension_zip = ".zip"
zip_file = filename + extension_zip

# !rm -rf $zip_file
!zip -r $zip_file {folders_or_files_to_save} # FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_file_to_drive(destination_name, path_to_file)

  adding: content/lightning_logs/ (stored 0%)
  adding: content/lightning_logs/version_0/ (stored 0%)
  adding: content/lightning_logs/version_0/events.out.tfevents.1699499053.8133a64222aa.909.0 (deflated 45%)
  adding: content/lightning_logs/version_0/hparams.yaml (stored 0%)
  adding: content/lightning_logs/version_1/ (stored 0%)
  adding: content/lightning_logs/version_1/events.out.tfevents.1699504523.8133a64222aa.909.2 (deflated 19%)
  adding: content/lightning_logs/version_1/events.out.tfevents.1699499829.8133a64222aa.909.1 (deflated 71%)
  adding: content/lightning_logs/version_1/hparams.yaml (stored 0%)
  adding: content/lightning_logs/version_1/checkpoints/ (stored 0%)
  adding: content/lightning_logs/version_1/checkpoints/epoch=11-step=6384.ckpt (deflated 8%)
File ID: 1w9YXjbe4iwTHeR7i5xMWJQJJxtb1C31R


{'id': '1w9YXjbe4iwTHeR7i5xMWJQJJxtb1C31R'}